In [ ]:
# Cell 1: Imports and basic setup
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, RocCurveDisplay

# Display options for cleaner outputs
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 200)

# Visualization style
sns.set_style('whitegrid')

print("All libraries imported successfully!")

In [ ]:
# Cell 2: Load CSV and get a quick overview
df = pd.read_csv('survey.csv')

# Show the dimensions of the dataset
print("Dataset shape:", df.shape)

# Display the first 6 rows
display(df.head(6))

# Display summary info about the dataset (column types, non-null counts, etc.)
display(df.info())

In [ ]:
# Cell 3: Exploratory Data Analysis (EDA)

# List all columns
print("Columns in the dataset:", list(df.columns))

# Show counts of each value in the target column ('treatment')
print("\nTarget value counts (treatment):")
print(df['treatment'].value_counts(dropna=False))

# Calculate percentage of missing values per column
missing_pct = df.isna().mean().sort_values(ascending=False) * 100
print("\nColumns with missing values (%):")
display(missing_pct[missing_pct > 0].round(2))

# Visualize the distribution of the target variable
plt.figure(figsize=(8,4))
sns.countplot(data=df, x='treatment', order=df['treatment'].value_counts().index)
plt.title('Distribution of the target: treatment')
plt.show()

# Visualize the distribution of age
plt.figure(figsize=(8,4))
sns.histplot(df['Age'].dropna(), bins=30)
plt.title('Age distribution')
plt.show()

In [ ]:
# Cell 4: Basic cleaning: Age, Gender normalization, and mapping Yes/No to 1/0
df2 = df.copy()  # work on a copy to preserve original data

# Age cleaning: convert to numeric and filter out unrealistic values

df2['Age'] = pd.to_numeric(df2['Age'], errors='coerce')  # convert to numeric, invalids become NaN
print("Before filtering: number of null ages =", df2['Age'].isna().sum())

# Optional: remove ages outside a realistic range (14-100)
df2.loc[(df2['Age'] < 14) | (df2['Age'] > 100), 'Age'] = np.nan
print("After filtering: number of null ages =", df2['Age'].isna().sum())


# Gender normalization: map variations to Male / Female / Other

def clean_gender(x):
    if pd.isna(x):
        return 'Other'
    s = str(x).strip().lower()
    # common male variants
    if s in ['male', 'm', 'man', 'male-ish', 'maile', 'mal', 'cis male', 'male (cis)']:
        return 'Male'
    # common female variants
    if s in ['female', 'f', 'woman', 'female (cis)', 'cis female']:
        return 'Female'
    # anything else (including 'trans' or 'non-binary') as Other
    return 'Other'

df2['Gender_clean'] = df2['Gender'].apply(clean_gender)


# Map binary Yes/No columns to 1/0

binary_cols = ['self_employed', 'family_history', 'treatment', 'remote_work', 'tech_company']

for col in binary_cols:
    if col in df2.columns:
        df2[col] = df2[col].map({'Yes': 1, 'No': 0})  # convert Yes/No to 1/0
        # keep existing numeric values as-is
        df2[col] = df2[col].fillna(df2[col]).infer_objects(copy=False)

print("Basic cleaning done. Sample data:")
display(df2[['Age','Gender','Gender_clean','self_employed','family_history','treatment']].head())

# Distribution of target variable
plt.figure(figsize=(8,4))
sns.countplot(data=df2, x='treatment', order=df2['treatment'].value_counts().index)
plt.title('Distribution of target: treatment')
plt.show()

# Age distribution
plt.figure(figsize=(8,4))
sns.histplot(df2['Age'].dropna(), bins=30)
plt.title('Age distribution')
plt.show()

In [ ]:
# Cell 5: Initial features and simple new features
df3 = df2.copy()  # work on a new copy to preserve previous cleaning

# Example of a binary feature: long_hours (e.g., work hours > 50)
# Note: if there is no 'hours' column in the dataset, we skip this part.
# Here, we assume there is no 'hours' column, so we do not create it.

# List of candidate features to consider for analysis/modeling
candidate_features = [
    'Age', 
    'Gender_clean',
    'self_employed',
    'family_history',
    'work_interfere',   # categorical: 'Never','Rarely','Sometimes','Often'
    'no_employees',     # categorical: company size
    'remote_work',
    'tech_company',
    'benefits',
    'care_options',
    'wellness_program',
    'seek_help',
    'anonymity'
]

# Keep only the features that exist in the current dataframe
candidate_features = [c for c in candidate_features if c in df3.columns]
print("Candidate features used:", candidate_features)

# Quick peek at unique values for categorical features (or those with <20 unique values)
for col in candidate_features:
    if df3[col].dtype == 'object' or df3[col].nunique() < 20:
        print(f"\n--- {col} unique values ---")
        print(df3[col].fillna('NA').value_counts().head(10))

In [ ]:
# Cell 6: Preprocessing Pipeline (!= Feature Engineering)

# Separate numerical and categorical features
num_features = [c for c in candidate_features if df3[c].dtype in ['int64','float64'] and c != 'treatment']
cat_features = [c for c in candidate_features if c not in num_features]

print("Numerical features:", num_features)
print("Categorical features:", cat_features)

# Numerical pipeline: impute missing values with median, then scale
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),  # fill missing values with median
    ('scaler', StandardScaler())                    # standardize (mean=0, std=1)
])

# Categorical pipeline: impute missing values with a constant, then one-hot encode
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Missing')),  # fill missing with 'Missing'
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)) # convert categories to 0/1 columns
])

# Combine pipelines using ColumnTransformer
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features)
], remainder='drop', sparse_threshold=0)

In [ ]:
# Cell 7: Prepare features (X) and target (y), then split into train/test sets

# Select features and target
X = df3[candidate_features].copy()
y = df3['treatment'].map({'Yes': 1, 'No': 0}) if df3['treatment'].dtype == 'object' else df3['treatment']

# Remove rows with missing target values
mask = y.notna()
X = X[mask]
y = y[mask].astype(int)

# Split data into training and test sets (80/20) with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,        # maintain target class distribution
    random_state=42
)

# Show shapes and target distribution in training set
print("Train/test shapes:", X_train.shape, X_test.shape)
print("Train target distribution:", np.bincount(y_train)/len(y_train))

In [ ]:
# Cell 8: Baseline Dummy model and Logistic Regression with cross-validation
# Dummy baseline: always predict the most frequent class
dummy = Pipeline([
    ('preproc', preprocessor),
    ('clf', DummyClassifier(strategy='most_frequent'))
])
dummy.fit(X_train, y_train)
print("Baseline (most frequent) test accuracy:", dummy.score(X_test, y_test))

# Logistic Regression with cross-validation
log_pipe = Pipeline([
    ('preproc', preprocessor),
    ('clf', LogisticRegression(
        max_iter=1000,          # allow enough iterations for convergence
        class_weight='balanced',# handle class imbalance
        solver='liblinear'      # suitable for small/medium datasets
    ))
])

# 5-fold stratified cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores_f1 = cross_val_score(log_pipe, X_train, y_train, cv=cv, scoring='f1')
print("Logistic Regression CV F1 score (mean ± std):", scores_f1.mean().round(4), "±", scores_f1.std().round(4))

# Fit on the full training set and evaluate on test set
log_pipe.fit(X_train, y_train)
y_pred = log_pipe.predict(X_test)

print("Logistic Regression Test Classification Report:")
print(classification_report(y_test, y_pred, digits=4))

In [ ]:
# Cell 9: Feature Importance Analysis (Original Logistic Regression Model)
# Get the original trained model from the pipeline
model_orig = log_pipe.named_steps['clf']

# Get the preprocessor
preprocessor_orig = log_pipe.named_steps['preproc']

# Get feature names
onehot_features_orig = preprocessor_orig.transformers_[1][1].named_steps['onehot'].get_feature_names_out(cat_features)
all_features_orig = num_features + list(onehot_features_orig)

# Create a DataFrame for coefficients
coefs_orig = model_orig.coef_[0]
feature_importance_orig = pd.DataFrame({
    'Feature': all_features_orig,
    'Coefficient': coefs_orig,
    'Abs_Coefficient': abs(coefs_orig)
})

# 5. Top 10 features by absolute impact
print("Top 10 most important features (overall impact) - Original Model:")
display(feature_importance_orig.sort_values(by='Abs_Coefficient', ascending=False).head(10))

# 6. Top 5 features indicating 'Treatment = Yes'
print("\nTop 5 features indicating 'Treatment = Yes':")
display(feature_importance_orig.sort_values(by='Coefficient', ascending=False).head(5))

# 7. Top 5 features indicating 'Treatment = No'
print("\nTop 5 features indicating 'Treatment = No':")
display(feature_importance_orig.sort_values(by='Coefficient', ascending=True).head(5))


In [ ]:
# Cell 10: Confusion Matrix (Original Model)
# 1. Generate the confusion matrix
# y_test: true labels, y_pred: predicted labels from the original model
cm_original = confusion_matrix(y_test, y_pred)

# 2. Create a display object for the confusion matrix
disp_original = ConfusionMatrixDisplay(
    confusion_matrix=cm_original, 
    display_labels=['No', 'Yes']  # adjust labels to match your target
)

# 3. Plot the confusion matrix
print("Confusion Matrix (Original Model - No Tuning):")
disp_original.plot(cmap=plt.cm.Blues, colorbar=False)
disp_original.ax_.grid(False)  # turn off grid
plt.show()

In [ ]:
# Cell 11: ROC Curve (Receiver Operating Characteristic) - Original Model
# Create the canvas
fig, ax = plt.subplots()

# Draw the ROC curve for the original logistic regression model
RocCurveDisplay.from_estimator(
    log_pipe,   # original model
    X_test,
    y_test,
    name='Logistic Regression (Original)',
    ax=ax
)

# Draw the "chance" line (random guess) on the same plot
ax.plot([0, 1], [0, 1], 'k--', label='Chance (AUC = 0.50)')

# Clean up and show
ax.set_title('ROC Curve - Original Logistic Regression Model')
ax.legend()
plt.show()

# Print the AUC score separately
y_probs_orig = log_pipe.predict_proba(X_test)[:, 1]  # Probabilities for the "Yes" class
auc_score_orig = roc_auc_score(y_test, y_probs_orig)
print(f"Area Under the Curve (AUC Score - Original Model): {auc_score_orig:.4f}")

In [ ]:
# Cell 12: Model Error Analysis (Original Logistic Regression Model)

# Make predictions on the test set using the original model
y_pred_orig = log_pipe.predict(X_test)

# Identify errors
errors_mask_orig = (y_pred_orig != y_test)
X_test_errors_orig = X_test[errors_mask_orig]
y_test_errors_orig = y_test[errors_mask_orig]
y_pred_errors_orig = y_pred_orig[errors_mask_orig]

print(f"Total errors: {errors_mask_orig.sum()} / {len(y_test)} ({errors_mask_orig.sum()/len(y_test)*100:.1f}%)")
print("\n" + "="*60)

# eparate false positives and false negatives
false_positives_orig = (y_pred_errors_orig == 1) & (y_test_errors_orig == 0)
false_negatives_orig = (y_pred_errors_orig == 0) & (y_test_errors_orig == 1)

print(f"\nFalse Positives: {false_positives_orig.sum()} cases (Predicted 'Yes' but was 'No')")
print(f"False Negatives: {false_negatives_orig.sum()} cases (Predicted 'No' but was 'Yes')")

# Profile False Positives
print("\n" + "="*60)
print("PROFILE OF FALSE POSITIVES:")
if false_positives_orig.sum() > 0:
    fp_data_orig = X_test_errors_orig[false_positives_orig]
    print("\nAverage Age:", fp_data_orig['Age'].mean())
    print("\nGender:")
    print(fp_data_orig['Gender_clean'].value_counts())
    print("\nWork Interfere:")
    print(fp_data_orig['work_interfere'].value_counts())

# Profile False Negatives
print("\n" + "="*60)
print("PROFILE OF FALSE NEGATIVES:")
if false_negatives_orig.sum() > 0:
    fn_data_orig = X_test_errors_orig[false_negatives_orig]
    print("\nAverage Age:", fn_data_orig['Age'].mean())
    print("\nGender:")
    print(fn_data_orig['Gender_clean'].value_counts())
    print("\nWork Interfere:")
    print(fn_data_orig['work_interfere'].value_counts())

# Visualization: compare distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Age histogram
axes[0].hist(X_test[~errors_mask_orig]['Age'].dropna(), bins=20, alpha=0.5, label='Correct', color='green')
axes[0].hist(X_test_errors_orig['Age'].dropna(), bins=20, alpha=0.5, label='Errors', color='red')
axes[0].set_xlabel('Age')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Age Distribution: Correct vs Errors')
axes[0].legend()

# Work Interfere bar plot
work_counts_correct_orig = X_test[~errors_mask_orig]['work_interfere'].value_counts()
work_counts_errors_orig = X_test_errors_orig['work_interfere'].value_counts()

x = range(len(work_counts_correct_orig))
width = 0.35
axes[1].bar([i - width/2 for i in x], work_counts_correct_orig.values, width, label='Correct', color='green', alpha=0.7)
axes[1].bar([i + width/2 for i in x], work_counts_errors_orig.values, width, label='Errors', color='red', alpha=0.7)
axes[1].set_xlabel('Work Interfere')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Work Interfere: Correct vs Errors')
axes[1].set_xticks(x)
axes[1].set_xticklabels(work_counts_correct_orig.index, rotation=45)
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Cell 13: Definition of the tuning pipeline

# Create the full pipeline
# Combine the preprocessor (already defined) with a Logistic Regression model
# Note: solver='liblinear' is good for small datasets and supports L1/L2 regularization
#       max_iter=1000 avoids convergence warnings
pipe_lr = Pipeline([
    ('pre', preprocessor), 
    ('model', LogisticRegression(solver='liblinear', max_iter=1000))
])

# Define the grid of parameters to test
# We will test different regularization strengths (C) and types of regularization (penalty)
# 'model__C' means: set the parameter 'C' in the pipeline step named 'model'
param_grid = {
    'model__penalty': ['l1', 'l2'],
    'model__C': [0.001, 0.01, 0.1, 1, 10, 100] 
}

# 3. Set up GridSearchCV
# cv=5 -> 5-fold cross-validation
# scoring='f1' -> optimize for F1 score
# n_jobs=-1 -> use all available CPU cores for faster computation
grid_lr = GridSearchCV(pipe_lr, param_grid, cv=5, scoring='f1', n_jobs=-1)

In [ ]:
# Cell 14: Run the hyperparameter tuning (GridSearch)

start_time = time.time()

# Fit the GridSearchCV on the training data
# It will try all combinations in param_grid using X_train and y_train
grid_lr.fit(X_train, y_train)

end_time = time.time()

# Print the time it took
print(f"GridSearch took {end_time - start_time:.2f} seconds.")
print("---")

# Print the best F1 score found during cross-validation
print(f"Best F1 score (from CV): {grid_lr.best_score_:.4f}")

# Print the combination of parameters that gave the best score
print("Best parameters found:")
print(grid_lr.best_params_)

In [ ]:
# Cell 15: Evaluate the optimized model on the test set
# Get the best estimator found by GridSearchCV
best_lr = grid_lr.best_estimator_

# Make predictions on the test set
y_pred_lr_tuned = best_lr.predict(X_test)

# Display the classification report
print("Classification Report (Optimized Model on Test Set):")
print(classification_report(y_test, y_pred_lr_tuned, digits=4))

In [ ]:
# Cell 16: Feature Importance Analysis

# Get the trained model from the pipeline
model = best_lr.named_steps['model']

# Get the preprocessor to retrieve feature names
preprocessor = best_lr.named_steps['pre']

# Get the feature names
# The preprocessor has two transformers ('num' and 'cat')
# The 'cat' transformer contains the one-hot encoder
onehot_features = preprocessor.transformers_[1][1].named_steps['onehot'].get_feature_names_out(cat_features)

# Combine numeric features + one-hot encoded categorical features
# (The order must match the ColumnTransformer: numeric first, then categorical)
all_features = num_features + list(onehot_features)

# Create a nice DataFrame to view results
coefs = model.coef_[0]
feature_importance = pd.DataFrame({
    'Feature': all_features,
    'Coefficient': coefs,
    'Abs_Coefficient': abs(coefs)  # absolute value to sort by importance
})

# Show the top 10 most impactful features (positive or negative)
print("Top 10 most important features (overall impact):")
display(feature_importance.sort_values(by='Abs_Coefficient', ascending=False).head(10))

# Show the top 5 features that most indicate 'Treatment = Yes'
print("\nTop 5 features that increase the likelihood of 'Treatment = Yes':")
display(feature_importance.sort_values(by='Coefficient', ascending=False).head(5))

# Show the top 5 features that most indicate 'Treatment = No'
print("\nTop 5 features that increase the likelihood of 'Treatment = No':")
display(feature_importance.sort_values(by='Coefficient', ascending=True).head(5))


In [ ]:
# Cell 17: Confusion Matrix
# Generate the matrix
cm = confusion_matrix(y_test, y_pred_lr_tuned)

# Draw the matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, 
                              display_labels=['No', 'Yes'])

# Show the plot
print("Confusion Matrix (Optimized Model):")

# We add 'colorbar=False' to remove the scale on the right
disp.plot(cmap=plt.cm.Blues, colorbar=False) 

# We access the plot's 'axis' (ax_) and turn off the grid
disp.ax_.grid(False)

plt.show()

In [ ]:
# Cell 18: ROC Curve (Receiver Operating Characteristic)
# First, we create the "canvas" (the axis)
fig, ax = plt.subplots()

# Now, we tell RocCurveDisplay to draw itself on that canvas
# It uses .predict_proba() automatically to get the "confidence"
RocCurveDisplay.from_estimator(
    best_lr, # tuned model from GridSearch
    X_test,
    y_test,
    name='Logistic Regression (Tuned)', # Name for the legend
    ax=ax # use the canvas we created
)

# Finally, we draw the "chance" (Dummy) line on the SAME canvas
ax.plot([0, 1], [0, 1], 'k--', label='Chance (AUC = 0.50)') # 'k--' is a black dashed line
    
# 4. Clean up and show
ax.set_title('ROC Curve (Optimized Model)')
ax.legend() # Activates the legend (which shows the model's AUC)
plt.show()

# 5. Print the AUC score separately
# The plot already shows it, but this way you have the number
y_probs = best_lr.predict_proba(X_test)[:, 1] # Probabilities for the "Yes" class only
auc_score = roc_auc_score(y_test, y_probs)
print(f"Area Under the Curve (AUC Score): {auc_score:.4f}")

In [ ]:
# Cell 19: Model Error Analysis
# Identify which predictions were wrong
errors_mask = (y_pred_lr_tuned != y_test)

# Get the features of the misclassified samples
X_test_errors = X_test[errors_mask]
y_test_errors = y_test[errors_mask]
y_pred_errors = y_pred_lr_tuned[errors_mask]

print(f"Total errors: {errors_mask.sum()} / {len(y_test)} ({errors_mask.sum()/len(y_test)*100:.1f}%)")
print("\n" + "="*60)

# Separate the two types of errors
false_positives = (y_pred_errors == 1) & (y_test_errors == 0)
false_negatives = (y_pred_errors == 0) & (y_test_errors == 1)

print(f"\nFalse Positives: {false_positives.sum()} cases")
print("   (Model predicted 'Yes' but it was 'No')")
print(f"\nFalse Negatives: {false_negatives.sum()} cases")
print("   (Model predicted 'No' but it was 'Yes')")

#  Analyze characteristics of false positives
print("\n" + "="*60)
print("PROFILE OF FALSE POSITIVES:")
print("="*60)
if false_positives.sum() > 0:
    fp_data = X_test_errors[false_positives]
    print("\nAverage Age:", fp_data['Age'].mean())
    print("\nGender:")
    print(fp_data['Gender_clean'].value_counts())
    print("\nWork Interfere:")
    print(fp_data['work_interfere'].value_counts())

# Analyze characteristics of false negatives
print("\n" + "="*60)
print("PROFILE OF FALSE NEGATIVES:")
print("="*60)
if false_negatives.sum() > 0:
    fn_data = X_test_errors[false_negatives]
    print("\nAverage Age:", fn_data['Age'].mean())
    print("\nGender:")
    print(fn_data['Gender_clean'].value_counts())
    print("\nWork Interfere:")
    print(fn_data['work_interfere'].value_counts())

# VISUALIZATION: Compare distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Age distribution
axes[0].hist(X_test[~errors_mask]['Age'].dropna(), bins=20, alpha=0.5, label='Correct', color='green')
axes[0].hist(X_test_errors['Age'].dropna(), bins=20, alpha=0.5, label='Errors', color='red')
axes[0].set_xlabel('Age')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Age Distribution: Correct vs Errors')
axes[0].legend()

# Plot 2: Work Interfere
work_counts_correct = X_test[~errors_mask]['work_interfere'].value_counts()
work_counts_errors = X_test_errors['work_interfere'].value_counts()

x = range(len(work_counts_correct))
width = 0.35
axes[1].bar([i - width/2 for i in x], work_counts_correct.values, width, label='Correct', color='green', alpha=0.7)
axes[1].bar([i + width/2 for i in x], work_counts_errors.values, width, label='Errors', color='red', alpha=0.7)
axes[1].set_xlabel('Work Interfere')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Work Interfere: Correct vs Errors')
axes[1].set_xticks(x)
axes[1].set_xticklabels(work_counts_correct.index, rotation=45)
axes[1].legend()

plt.tight_layout()
plt.show()
